In [2]:
using ITensors, ITensorMPS
using LinearAlgebra
using Plots
using Random
using Statistics

# --- FIX 1: Disable the tensor order warning for N >= 7 ---
ITensors.disable_warn_order()

# --- Configuration ---
N_range = 3:13
σ_disorder = 0.002

# Function to generate Gaussian random weights
function generate_couplings(N, σ)
    J = zeros(Float64, N, N)
    for i in 1:N
        for j in i+1:N
            weight = randn() * σ
            J[i, j] = weight
            J[j, i] = weight
        end
    end
    return J
end

overlaps = Float64[]
errors = Float64[]

println("Starting comparison between DMRG and ED...")

for N in N_range
    println("Processing N = $N ...")
    
    # 1. Define Sites
    sites = siteinds("S=1/2", N)
    
    # 2. Construct Hamiltonian
    J_mat = generate_couplings(N, σ_disorder)
    os = OpSum()
    for i in 1:N
        for j in i+1:N
            val = J_mat[i, j]
            # XXZ Terms: J(SxSx + SySy + SzSz)
            os += val, "Sx", i, "Sx", j
            os += val, "Sy", i, "Sy", j
            os += val, "Sz", i, "Sz", j
        end
    end
    H = MPO(os, sites)

    # 3. Solve with DMRG
    # Note: Increased 'nsweeps' and 'maxdim' slightly to ensure convergence for N=6+
    psi0 = randomMPS(sites, 20)
    energy_dmrg, psi_dmrg = dmrg(H, psi0; 
                                 nsweeps=40, 
                                 maxdim=[10, 20, 100, 200, 400], 
                                 cutoff=1E-12, 
                                 outputlevel=0)

    # 4. Solve with Exact Diagonalization (ED)
    # Contract the MPO into a single dense tensor
    H_dense = prod(H) 
    
    # Create the dense matrix with explicit index ordering
    # prime.(sites) are rows, sites are columns
    H_mat = Array(H_dense, prime.(sites)..., sites...) 
    H_mat = reshape(H_mat, 2^N, 2^N) 
    
    # Diagonalize
    evals, evecs = eigen(Hermitian(H_mat))
    psi_ed_vec = evecs[:, 1] 
    
    # 5. Calculate Overlap
    # Contract DMRG MPS to dense tensor
    psi_dmrg_dense = prod(psi_dmrg)
    # Ensure vector uses same site ordering as the matrix columns
    psi_dmrg_vec = reshape(Array(psi_dmrg_dense, sites...), 2^N)
    
    # Calculate overlap
    ovlp = abs(dot(conj(psi_dmrg_vec), psi_ed_vec))
    
    push!(overlaps, ovlp)
    
    # Note: Your report (Fig 6) plots the 'Overlap', but the text mentions 'Error'.
    # If you want to replicate the 'dip' exactly as in your report, plot Overlap.
    # If you want 'Error', use (1.0 - ovlp).
    println("  N=$N | Overlap = $ovlp")
end

# --- Plotting ---
p = plot(N_range, overlaps,
    title = "Overlap of DMRG and Exact Diagonalization",
    xlabel = "Number of Nodes",
    ylabel = "Overlap",
    legend = false,
    marker = :circle,
    color = :blue,
    linewidth = 2,
    framestyle = :box,
    grid = true
)

savefig(p, "DMRG_ED_Overlap_Plot.png")
println("Plot saved to DMRG_ED_Overlap_Plot.png")

Starting comparison between DMRG and ED...
Processing N = 3 ...
  N=3 | Overlap = 0.8926273285154122
Processing N = 4 ...
  N=4 | Overlap = 1.0
Processing N = 5 ...
  N=5 | Overlap = 0.09430268280947843
Processing N = 6 ...
  N=6 | Overlap = 0.7991138912840758
Processing N = 7 ...
  N=7 | Overlap = 0.722043788133101
Processing N = 8 ...
  N=8 | Overlap = 0.6142801588135876
Processing N = 9 ...
  N=9 | Overlap = 0.8463548425969657
Processing N = 10 ...
  N=10 | Overlap = 0.7261326491060807
Processing N = 11 ...
  N=11 | Overlap = 0.8683717515706377
Processing N = 12 ...
  N=12 | Overlap = 0.2710742727022132
Processing N = 13 ...
  N=13 | Overlap = 0.3929611792439572
Plot saved to DMRG_ED_Overlap_Plot.png
